In [1]:
import pandas as pd

In [4]:
df = pd.read_excel('./question_architecture/SCoRE.xls')
df

,大分類,小分類,キーワード,レベル,用例,日本語訳
0,代名詞,指示代名詞,that,beginner,The footstep was {that} of a teacher.,その足あとは先生のものでした。
1,代名詞,指示代名詞,that,beginner,His voice is louder than {that} of mine.,彼の声は私の声よりも大きいです。
2,代名詞,指示代名詞,that,beginner,My room is bigger than {that} of John's.,私の部屋はジョンの部屋より広いです。
3,代名詞,指示代名詞,that,beginner,His house is connected to {that} of Linda's.,彼の家はリンダの家とつながっています。
4,代名詞,指示代名詞,that,beginner,The voice was {that} of a singer.,その声はある歌手のものでした。
...,...,...,...,...,...,...
15623,天気,天気,windy,beginner,It's too {windy} to sit outside.,外で座るには風が強すぎます。
15624,天気,天気,windy,beginner,{Windy} days are great for flying kites.,風のある日は凧あげに最高です。
15625,天気,天気,windy,beginner,"Chicago is called the ""{windy} city.""",シカゴは「風の街」と呼ばれています。
15626,天気,天気,windy,beginner,We're expecting {windy} conditions.,私たちは風の強い状態を予期しているところです。


In [53]:
import re
# 文書ごとのリストを作る
sentences = []
japanese = []
# {}を取り除く
for sentence, jp in zip(df['用例'].values, df['日本語訳'].values):
    sentences.append(''.join(re.split('[{}]', sentence)))
    japanese.append(jp)
print(sentences[:5], japanese[:5])

['The footstep was that of a teacher.', 'His voice is louder than that of mine.', "My room is bigger than that of John's.", "His house is connected to that of Linda's.", 'The voice was that of a singer.'] ['その足あとは先生のものでした。', '彼の声は私の声よりも大きいです。', '私の部屋はジョンの部屋より広いです。', '彼の家はリンダの家とつながっています。', 'その声はある歌手のものでした。']


In [46]:
import nltk
import itertools

# 文書ごとに分かち書き
word = []
for text in sentences:
    word.append(nltk.word_tokenize(text))
words = list(itertools.chain.from_iterable(word))

# 単語ごとの品詞の取得
pos = nltk.pos_tag(words)

In [47]:
len(pos)

153906

,word,pos_tag
0,The,DT
1,footstep,NN
2,was,VBD
3,that,IN
4,of,IN
...,...,...
153901,be,VB
153902,a,DT
153903,windy,JJ
153904,night,NN


In [54]:
import random
import re
import itertools
import pandas as pd
import nltk
import spacy
import requests
from bs4 import BeautifulSoup

# コーパスからテキスト取得
df = pd.read_excel('SCoRE.xls')

# 文書ごとのリストを作る
sentences = []
japanese = []
# {}を取り除く
for sentence, jp in zip(df['用例'].values, df['日本語訳'].values):
    sentences.append(''.join(re.split('[{}]', sentence)))
    japanese.append(jp)
    
# 文書ごとに分かち書き
word = []
for text in sentences:
    word.append(nltk.word_tokenize(text))
words = list(itertools.chain.from_iterable(word))

# 単語ごとの品詞の取得
pos = nltk.pos_tag(words)

# 単語と品詞がペアになったDataFrameの作成
pos_df = pd.read_csv('word_pos.csv')
    
    
# 問題    

print('\n次の文章の空欄に当てはまる単語を選択肢の中から選び、数字で答えなさい。')

cnt = 0
correct_num = 0
while(1):
    cnt+=1
    import random
    # 問題となる文をランダムに取得
    question_sent_idx = random.choice(range(len(sentences)))
    question_sent = sentences[question_sent_idx]
    japanese_sent = japanese[question_sent_idx]
    

    while(1):
        # 問題文を単語ごとに分割
        question_sent_words = nltk.word_tokenize(question_sent)
        # 問題文を単語ごとに分割した時のリストからランラムにインデックス番号を取得
        question_word_index = random.choice(range(len(nltk.word_tokenize(question_sent))))
        # 取得したインデックス番号に対応する単語を取得
        question_word = question_sent_words[question_word_index]

        # 問題文のそれぞれの単語の品詞を取得
        question_sent_pos = nltk.pos_tag(question_sent_words)

        # もし、選択可能でない品詞のものならもう一度実行し直す
        if question_sent_pos[question_word_index][1] in ['NNP', 'NNPS', 'FW', 'LS', 'SYM', ',', '.']:
            continue
        # それ以外なら、answerとして答えを変数として格納する
        else:
            answer = question_word
            question_pos = question_sent_pos[question_word_index][1]
            break

    # 問題文の作成
    question_sent_words[question_word_index] = ' ____ '
    question = ' '.join(question_sent_words)
    print(f'Question{cnt}: {japanese_sent}')
    print(question)
    print('----------------------------------------------------------')


    # 同じ品詞のものを取得
    while(1):
        wrong_answer = random.sample(pos_df.loc[pos_df['pos_tag'] == question_pos, 'word'].values.tolist(), 3)
        def has_duplicates(seq):
            return len(seq) != len(set(seq))
        if answer in wrong_answer:
            continue
        elif has_duplicates([str.lower(x) for x in wrong_answer]):
            continue
        else:
            break
    answers = wrong_answer
    answers.append(answer)
    answers_list = random.sample(answers, len(answers))
    for i, ans in enumerate(answers_list):
        print(f'{i+1}: {ans}')

    response = int(input('Which number is Correct?'))
    if answers_list[response-1] == answer:
        correct_num += 1
        print('Correct!!')
    else:
        print('Wrong!!')
        print(f'Correct answer is: {answer}')

    print('----------------------------------------------------------')
    if cnt % 10 == 0:
        is_continue = input('まだ続けますか？(y/n)')
        if is_continue == 'y':
            continue
        else:
            print('正答率: ', (correct_num / cnt) * 100, '%')
            break



次の文章の空欄に当てはまる単語を選択肢の中から選び、数字で答えなさい。
Question1: ギリシアの英雄ユリシーズはトロイの人びとに致命的となる木馬を送りました。
The  ____  hero Ulysses gave the Trojans a deadly wooden horse .
----------------------------------------------------------
1: popular
2: Arabian
3: plastic
4: Greek
Correct!!
----------------------------------------------------------
Question2: その殺人の裁判は多くの報道機関によって報道されました。
The murder trial was  ____  by many news agencies .
----------------------------------------------------------
1: been
2: rushed
3: covered
4: closed
Wrong!!
Correct answer is: covered
----------------------------------------------------------
Question3: 私はタイ航空の飛行機に乗っているところです。 
I  ____  flying on Thai Airlines .
----------------------------------------------------------
1: are
2: remind
3: call
4: 'm
Correct!!
----------------------------------------------------------
Question4: 私に彼女のものを手渡してもらえますか？
Would you  ____  me hers ?
----------------------------------------------------------
1: encyclopedia
2: hand
3: anyone
4: city
Correct!!

ValueError: invalid literal for int() with base 10: ''